In [ ]:
%load_ext pretty_jupyter

In [ ]:
import sys
import scvi
import cellxgene_census
import numpy as np
import pandas as pd
import scanpy as sc
import tomlkit
import logging
from os import path, system
from matplotlib import rc_context
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.ERROR)


In [ ]:
scGPT_model = "../resources/scGPT_CP/"
scGPT_index = "../resources/CellXGene_faiss_index/"
scGPT_column = "scGPT"

In [ ]:
## Pipeline parameters
with open("../config.toml", "r") as f:
    config = tomlkit.parse(f.read())

In [ ]:
ROOT_DIR = config["basic"]["ANALYSIS_DIR"]
DIR_SAVE = path.join(ROOT_DIR, config["basic"]["DIR_SAVE"])
TISSUE = config["basic"]["TISSUE"]
ANNOTATION_METHOD = config["annotation"]["ANNOTATION_METHOD"]
COUNTS_LAYER = config["normalization"]["COUNTS_LAYER"]
NORMAMALIZATION_LAYER = config["normalization"]["NORMALIZATION_METHOD"]
ANNOTATION_METHOD = config["annotation"]["ANNOTATION_METHOD"]

In [ ]:
adata = sc.read_h5ad(path.join(DIR_SAVE, "adata.h5ad"))

# scGPT

In [ ]:
# Run the scGPT annotation in a seperate process to avoid blocking the GPU resources in interactive mode
if ANNOTATION_METHOD == "scGPT":
    system(f"python utils/scgpt_annotate.py -i {path.join(DIR_SAVE, 'adata.h5ad')} -c {scGPT_column} -m {scGPT_model} -n {scGPT_index} -o {path.join(DIR_SAVE, 'adata.h5ad')}")
    adata = sc.read_h5ad(path.join(DIR_SAVE, "adata.h5ad"))
    with rc_context({"figure.figsize": (8,8)}):
        sc.pl.umap(adata, color=[scGPT_column])
        plt.tight_layout()
